In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
from mlflow.tracking import MlflowClient

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(42)

plt.rcParams['figure.figsize'] = [10, 7]
pd.set_option("max_columns", None)

## Load the csv file and add a device_id column to simulate having multiple IoT devices

In [ ]:
df = pd.read_csv('../data/power_plants.csv')
df.head()

- We want to simulate the fact of using multiple models in production at the same time (one model per IoT device). To do that, we can either split our data to 4 dataframes (1 dataframe per IoT device) or we can create a new column `device_id` where we specify the IoT device the record belongs to. We will go with the latter approach by assigning a random number between 1 and 4.

In [ ]:
import numpy as np

df["device_id"] = np.random.randint(1, 5, size=len(df))
df.head()

In [ ]:
df.device_id.value_counts()

# Model training

## Train model function

Create a function that trains a model for a single device. It should take the device dataframe as input an do the following steps:
- Split the data into training and test set
- Trains a `RandomForestRegressor`
- Evaluates it with `MAE`, `MSE` and `RMSE`
- Returns a dictionary with the following informations
    - `device_id`
    - `test_mae`
    - `test_mse`
    - `test_rmse`

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_model_for_an_iot_device(device_df: pd.DataFrame) -> dict:
    """Trains a model for a single device"""
    device_id = device_df.iloc[0].device_id
    
    # Split data
    X = device_df[["AT", "V", "AP", "RH"]]
    y = device_df["PE"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Fit model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    
    test_mae = mean_absolute_error(y_test, y_pred)
    test_mse = mean_squared_error(y_test, y_pred)
    test_rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    ret_dict = {
        "device_id": device_id,
        "test_mae": test_mae,
        "test_mse": test_mse,
        "test_rmse": test_rmse
    }
    return ret_dict

## Train a model for all the devices

Now that we can train a model for a given IoT device, let's orchestrate the training for all device models given the full dataset (sensor measures from all the IoT devices).

Create a function `train_models_for_all_iot_devices` that will take the full data and call the `train_model_for_an_iot_device` function with the corresponding device data. This function shouldd return a dataframe where each row corresponds to each device model training metadata (`device_id`, `fitted_model`, `test_mae`, `test_mse`, `test_rmse`).

In [ ]:
def train_models_for_all_iot_devices(data: pd.DataFrame) -> pd.DataFrame:
    ret_dict_list = []
    for device_id in data.device_id.unique():
        iot_device_df = data[data.device_id == device_id]
        ret_dict_list.append(train_model_for_an_iot_device(iot_device_df))
    return ret_dict_list

In [ ]:
train_models_for_all_iot_devices(df)

## Mlflow integration

`mlflow server --backend-store-uri sqlite:////tmp/mlruns.db --default-artifact-root /tmp/mlruns`

In [ ]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [ ]:
experiment_name = "Iot device model training"

mlflow.set_experiment(experiment_name)

experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
experiment_id

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_model_for_an_iot_device(device_df: pd.DataFrame) -> dict:
    """Trains a model for a single device"""
    device_id = device_df.iloc[0].device_id
    
    with mlflow.start_run():
        mlflow.sklearn.autolog()

        # Split data
        X = device_df[["AT", "V", "AP", "RH"]]
        y = device_df["PE"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        mlflow.log_params(
            {"nb_sample": len(X), "nb_training_samples": len(X_train), "nb_testing_samples": len(X_test)})

        # Fit model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)

        test_mae = mean_absolute_error(y_test, y_pred)
        test_mse = mean_squared_error(y_test, y_pred)
        test_rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metrics({"test_mae": test_mae, "test_mse": test_mse, "test_rmse": test_rmse})

        ret_dict = {
            "device_id": device_id,
            "test_mae": test_mae,
            "test_mse": test_mse,
            "test_rmse": test_rmse
        }
    return ret_dict

In [ ]:
def train_models_for_all_iot_devices(data: pd.DataFrame) -> pd.DataFrame:
    ret_dict_list = []
    for device_id in data.device_id.unique():
        iot_device_df = data[data.device_id == device_id]
        ret_dict_list.append(train_model_for_an_iot_device(iot_device_df))
    return ret_dict_list

In [ ]:
train_models_for_all_iot_devices(df)

## Train a model each month

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_model_for_an_iot_device(device_df: pd.DataFrame, month_name: str) -> dict:
    """Trains a model for a single device"""
    device_id = int(device_df.iloc[0].device_id)
    
    with mlflow.start_run(nested=True, run_name=f"device {device_id}"):
        mlflow.log_params({"device_id": device_id, "month": month_name})
        
        
        mlflow.sklearn.autolog()

        # Split data
        X = device_df[["AT", "V", "AP", "RH"]]
        y = device_df["PE"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        mlflow.log_params(
            {"nb_sample": len(X), "nb_training_samples": len(X_train), "nb_testing_samples": len(X_test)})

        # Fit model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)

        test_mae = mean_absolute_error(y_test, y_pred)
        test_mse = mean_squared_error(y_test, y_pred)
        test_rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metrics({"test_mae": test_mae, "test_mse": test_mse, "test_rmse": test_rmse})

        ret_dict = {
            "device_id": device_id,
            "test_mae": test_mae,
            "test_mse": test_mse,
            "test_rmse": test_rmse
        }
    return ret_dict

In [ ]:
def train_models_for_all_iot_devices(data: pd.DataFrame, month_name: str) -> pd.DataFrame:
    with mlflow.start_run(run_name=month_name):
        ret_dict_list = []
        for device_id in data.device_id.unique():
            iot_device_df = data[data.device_id == device_id]
            ret_dict = train_model_for_an_iot_device(iot_device_df, month_name)
            ret_dict_list.append(ret_dict)
    return ret_dict_list

In [ ]:
train_models_for_all_iot_devices(df, "june")

In [ ]:
train_models_for_all_iot_devices(df, "july")

In [ ]:
train_models_for_all_iot_devices(df, "august")

# Simulate drift in the data

In [ ]:
df_lower_distribution = df[df["PE"] < 450]

In [ ]:
# Create 2 "months" worth of data from the same distribution
indexs = np.random.rand(len(df_lower_distribution))
indexs

In [ ]:
mask = indexs < 0.5
df_march, df_april = df_lower_distribution[mask], df_lower_distribution[~mask]

In [ ]:
df_march["PE"].hist(bins=100)

In [ ]:
df_april["PE"].hist(bins=100)

In [ ]:
df_may = df[df["PE"] >= 450]
df_may["PE"].hist(bins=100)

# Train models for the different months

In [ ]:
_ = train_models_for_all_iot_devices(df_march, "march")

## Register a model in the Model Registry

In [ ]:
all_experiment_runs_df = mlflow.search_runs(experiment_id)
all_experiment_runs_df.head(2)

- Register model in Mlflow ModelRegistry

In [ ]:
device_id = 1
current_month = "march"
best_model_for_device_for_current_month_run_id = mlflow.search_runs(
    experiment_id,
    order_by=['metrics.test_rmse desc'],
    filter_string=f"params.device_id = '{device_id}' AND params.month = '{current_month}'"
).iloc[0]["run_id"]
best_model_for_device_for_current_month_run_id

In [ ]:
model_uri = f"runs:/{best_model_for_device_for_current_month_run_id}/model"
model_name_in_model_registry = f"powerplant_device_{device_id}"
model_version = mlflow.register_model(model_uri=model_uri, name=model_name_in_model_registry)

In [ ]:
model_version

In [ ]:
type(experiment_id)

In [ ]:
def register_iot_device_model_with_best_rmse(month: str, device_id: int, experiment_name: str):
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    run_id = mlflow.search_runs(
        experiment_id,
        order_by=['metrics.test_rmse desc'],
        filter_string=f"params.device_id = '{device_id}' AND params.month = '{month}'"
    ).iloc[0]["run_id"]
    
    model_name_in_model_registry = f"powerplant_device_{device_id}"
    model_version = mlflow.register_model(model_uri=f"runs:/{run_id}/model", name=model_name_in_model_registry)
    return model_version

In [ ]:
model_version = register_iot_device_model_with_best_rmse("march", 2, experiment_name)
model_version

## Transition model for a different stage

In [ ]:
client = MlflowClient()

In [ ]:
help(client.transition_model_version_stage)

In [ ]:
target_stage = "Production"
model_version = client.transition_model_version_stage(
    name=model_version.name, version=model_version.version, stage=target_stage
)
model_version

In [ ]:
def transition_model_to_a_new_stage(model_version, stage: str, archive_existing_versions: bool = True):
    valid_stage_values = ["staging", "production", "archived", "none"]
    assert stage in valid_stage_values, f"Invalid stage: {stage}. Valid stage values = {valid_stage_values}"
    print(model_version)
    client = MlflowClient()
    updated_model_version = client.transition_model_version_stage(
        name=model_version.name, version=model_version.version,
        stage=stage, archive_existing_versions=archive_existing_versions
    )
    return updated_model_version

In [ ]:
transition_model_to_a_new_stage(model_version, "staging")

In [ ]:
model_version = register_iot_device_model_with_best_rmse("march", 2, experiment_name)
model_version = transition_model_to_a_new_stage(model_version, "staging")
###### Human in the loop: Model testing and validation by the product owner
model_version = transition_model_to_a_new_stage(model_version, "production")

In [ ]:
model_version = transition_model_to_a_new_stage(model_version, "archived", False)

# Final pipeline

## March

In [ ]:
month = "March"

In [ ]:
# Train models
for _ in range(2):
    _ = train_models_for_all_iot_devices(df_march, month)

In [ ]:
# Register the model with the best RMSE in the model registry
device_1_model_version = register_iot_device_model_with_best_rmse(month, 1, experiment_name)
device_2_model_version = register_iot_device_model_with_best_rmse(month, 2, experiment_name)
device_3_model_version = register_iot_device_model_with_best_rmse(month, 3, experiment_name)
device_4_model_version = register_iot_device_model_with_best_rmse(month, 4, experiment_name)

In [ ]:
# Transition models to production
device_1_model_version = transition_model_to_a_new_stage(device_1_model_version, "production")
device_2_model_version = transition_model_to_a_new_stage(device_2_model_version, "production")
device_3_model_version = transition_model_to_a_new_stage(device_3_model_version, "production")
device_4_model_version = transition_model_to_a_new_stage(device_4_model_version, "production")

## April

In [ ]:
month = "April"

In [ ]:
# Train models
for _ in range(2):
    _ = train_models_for_all_iot_devices(df_april, month)

In [ ]:
# Register the model with the best RMSE in the model registry
device_1_model_version = register_iot_device_model_with_best_rmse(month, 1, experiment_name)
device_2_model_version = register_iot_device_model_with_best_rmse(month, 2, experiment_name)
device_3_model_version = register_iot_device_model_with_best_rmse(month, 3, experiment_name)
device_4_model_version = register_iot_device_model_with_best_rmse(month, 4, experiment_name)

In [ ]:
# Transition models to staging
stage = "staging"
device_1_model_version = transition_model_to_a_new_stage(device_1_model_version, stage)
device_2_model_version = transition_model_to_a_new_stage(device_2_model_version, stage)
device_3_model_version = transition_model_to_a_new_stage(device_3_model_version, stage)
device_4_model_version = transition_model_to_a_new_stage(device_4_model_version, stage)

In [ ]:
###### Human in the loop: Model testing and validation by the product owner
# Transition models to staging
stage = "production"
device_1_model_version = transition_model_to_a_new_stage(device_1_model_version, stage)
device_2_model_version = transition_model_to_a_new_stage(device_2_model_version, stage)
device_3_model_version = transition_model_to_a_new_stage(device_3_model_version, stage)
device_4_model_version = transition_model_to_a_new_stage(device_4_model_version, stage)

## May